In [1]:
!pip install -q transformers accelerate trl bitsandbytes datasets evaluate
!pip install -q peft scikit-learn

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

2024-06-26 18:38:37.773561: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-26 18:38:37.773703: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-26 18:38:37.897040: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
!huggingface-cli login --token  #enter your api token here

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [46]:

model_name = "FacebookAI/roberta-large"  #"meta-llama/Meta-Llama-3-8B" 


dataset_name = "sentence-transformers/stsb"


new_model = "Llama-3-8b-chat-finetune"


In [55]:
import pandas as pd
df = pd.read_csv('/kaggle/input/aaaaaaaaaaaaaa/kw_tfidf.csv')

In [56]:
df2 = df.iloc[:,[2,3]]

In [57]:
from datasets import Dataset
dataset = Dataset.from_pandas(df2)

In [58]:
# Load dataset (you can process it here)
print(dataset)

Dataset({
    features: ['full_text', 'score'],
    num_rows: 17307
})


In [59]:
def preprocess_function(examples):
    return {
        'input_text': [s1 for s1 in (examples['full_text'])],
        'label': [ float(score) for score in examples['score']]
    }

# Apply the preprocessing function to the dataset
pdataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/17307 [00:00<?, ? examples/s]

In [60]:
main_dataset = pdataset.remove_columns(['full_text', 'score'])

In [61]:
main_dataset

Dataset({
    features: ['input_text', 'label'],
    num_rows: 17307
})

In [11]:
traind = main_dataset['train']

In [12]:
evald = main_dataset['validation']

In [47]:


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)



model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    #quantization_config=bnb_config,
    device_map='auto',
    num_labels=1
)


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
print(model)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [62]:
from torch.utils.data import DataLoader

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    lora_alpha=8,
    lora_dropout=0.05,
    target_modules = [
        f"roberta.encoder.layer.{i}.attention.self.query" for i in range(24)
    ] + [
        f"roberta.encoder.layer.{i}.attention.self.key" for i in range(24)
    ] + [
        f"roberta.encoder.layer.{i}.attention.self.value" for i in range(24)
    ] + [
        f"roberta.encoder.layer.{i}.attention.output.dense" for i in range(24)
    ] + [
        f"roberta.encoder.layer.{i}.intermediate.dense" for i in range(24)
    ] + [
        f"roberta.encoder.layer.{i}.output.dense" for i in range(24)
    ],
    r=16,
    bias="none",
    task_type="SEQ_CLS",
)

#model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)



In [63]:

tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1

In [64]:
def data_preprocesing(row):
    return tokenizer(row['input_text'], truncation=True, max_length=512)

tokenized_data = main_dataset.map(data_preprocesing, batched=True,
remove_columns=['input_text'])
tokenized_data.set_format("torch")

from transformers import DataCollatorWithPadding

collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/17307 [00:00<?, ? examples/s]

In [65]:
print(tokenized_data)

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 17307
})


In [53]:


# Optional: Print to check the updated DatasetDict
print(tokenized_data)

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 17307
})


In [ ]:
training_args = TrainingArguments(
    output_dir = 'sentiment_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    num_train_epochs = 1,
    logging_steps=1,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    report_to="none"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_data,
    data_collator = collate_fn,
    args=training_args,
)

# Train model
rs = trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 512
  warnings.warn(


Epoch,Training Loss,Validation Loss


In [46]:
model.save_pretrained('/kaggle/working/')

# Save the tokenizer
tokenizer.save_pretrained('/kaggle/working/')

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('/kaggle/working/tokenizer_config.json',
 '/kaggle/working/special_tokens_map.json',
 '/kaggle/working/vocab.json',
 '/kaggle/working/merges.txt',
 '/kaggle/working/added_tokens.json',
 '/kaggle/working/tokenizer.json')

In [ ]:
import pandas as pd
df_test = pd.Dataframe(main_dataset['test'])

In [15]:
def generate_predictions(model):
    sentences = main_dataset['validation']['input_text']
    batch_size = 25
    all_outputs = []

    for i in range(0, len(sentences), batch_size):

        batch_sentences = sentences[i:i + batch_size]

        inputs = tokenizer(batch_sentences, return_tensors="pt", 
        padding=True, truncation=True, max_length=512)

        inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') 
        for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            all_outputs.append(outputs['logits'])
        
    final_outputs = torch.cat(all_outputs, dim=0)
    x=final_outputs.cpu().numpy()
    return x

x=generate_predictions(model)

In [45]:
x

array([[7.068718],
       [2.249843]], dtype=float32)

In [23]:
y[:10]

array([9.32207  , 8.889901 , 9.366098 , 4.923479 , 5.90688  , 5.7475395,
       9.221779 , 6.83445  , 8.749601 , 8.910006 ], dtype=float32)

In [22]:
(main_dataset['validation']['label'])[:10]

[10.0, 9.5, 10.0, 4.8, 5.5, 5.2, 10.0, 4.699999999999999, 7.5, 10.0]

In [19]:
z

array([-0.67792988, -0.61009884, -0.6339016 , ..., -1.37710643,
        2.0892992 ,  3.18048811])